## Example Bulk Si sp3s
### Symmetry adapted linear combinations
This example is essentially the same as the example of bulk silicon sp3 but include one additional orbitals, the higher 3s orbital, which is often included in the tight-binding parameterization of Si. 

Compare to the previous example ($3\times A_1$, $E (2-fold)$, $T_1 (3-fold)$, $4 \times T_2 (3-fold)$), the 4 additional $3s$ orbital form $2\times A_1$ and $T_2$. 

In [ ]:
from automaticTB.examples.structures import get_Si_structure_2s2p3s
from automaticTB.functions import get_namedLCs_from_nncluster

bulksi = get_Si_structure_2s2p3s()
si_environment = bulksi.nnclusters[0]
named_lcs = get_namedLCs_from_nncluster(si_environment)
for nlc in named_lcs:
    print(f"\"{nlc.name}\"")
    #print(nlc.lc)

### Free values of interaction
Additional interaction is given when we increase the number of atomic orbitals related to the introduction of the $3s$ atomic orbitals.

In [ ]:
from automaticTB.functions import get_free_AOpairs_from_nncluster_and_namedLCs
from automaticTB.printing import print_ao_pairs

free_pairs = get_free_AOpairs_from_nncluster_and_namedLCs(si_environment, named_lcs)

print_ao_pairs(
    bulksi, free_pairs
)